In [1]:
# Preparing the notebook by importing necessary modules
# SK Learn Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier, LogisticRegression, RidgeClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
# Standard imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from math import sqrt
# Import wrangle functions
from model import model_df, split_separate_scale
# Formatting my notebook
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
df= model_df()

In [3]:
df.head()

,dept,call_reason,source_id,council_district,resolution_days_due,level_of_delay,district_0,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,animal_care_services,code_enforcement_services,customer_services,development_services,metro_health,parks_and_rec,solid_waste_management,trans_and_cap_improvements,unknown_dept,buildings,business,cleanup,code,customer_service,field,land,license,misc,storm,streets,trades,traffic,waste,web_portal,311_mobile_app,constituent_call,internal_services_requests
CASEID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1014281655,Code Enforcement Services,code,Internal Services Requests,6,131,Extremely Late Response,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1014281665,Code Enforcement Services,code,Internal Services Requests,6,131,Extremely Late Response,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1014281677,Code Enforcement Services,code,Internal Services Requests,6,131,Extremely Late Response,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1014300321,Development Services,buildings,Constituent Call,4,933,Very Early Response,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1014326068,Code Enforcement Services,code,Internal Services Requests,4,131,Extremely Late Response,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [4]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = split_separate_scale(df, stratify_by= 'level_of_delay')

In [5]:
print("train: ", X_train.shape, ", validate: ", X_validate.shape, ", test: ", X_test.shape)
print("train: ", y_train.shape, ", validate: ", y_validate.shape, ", test: ", y_test.shape)

train:  (199760, 43) , validate:  (85612, 43) , test:  (71343, 43)
train:  (199760,) , validate:  (85612,) , test:  (71343,)


# Make Baseline Prediction

In [6]:
#Making a baseline prediction
df.level_of_delay.value_counts()

Early Response              264640
On Time Response             51531
Very Early Response          27320
Late Response                 8938
Very Late Response            3148
Extremely Late Response        991
Extremely Early Response       147
Name: level_of_delay, dtype: int64

In [7]:
# set up y_train, y_validate, and y_test as dataframes
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)

In [8]:
#concatenating the baseline prediction to the y_train
y_train['baseline'] = 'Early Response'

In [9]:
y_train.head()

,level_of_delay,baseline
CASEID,,
1016517770,Early Response,Early Response
1016203792,Early Response,Early Response
1016771848,Early Response,Early Response
1016634860,Early Response,Early Response
1016302671,On Time Response,Early Response


In [10]:
# calculate accuracy of baseline

print(' Baseline Accuracy: {:.2%}'.format(accuracy_score(y_train.level_of_delay, y_train.baseline)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.baseline, y_train.level_of_delay))
print('---')
print(classification_report(y_train.level_of_delay, y_train.baseline))

 Baseline Accuracy: 74.19%
---
Confusion Matrix
level_of_delay  Extremely Late Response  Very Late Response  Late Response  \
baseline                                                                     
Early Response                      555                1763           5005   

level_of_delay  On Time Response  Early Response  Very Early Response  \
baseline                                                                
Early Response             28857          148198                15299   

level_of_delay  Extremely Early Response  
baseline                                  
Early Response                        83  
---
                          precision    recall  f1-score   support

          Early Response       0.74      1.00      0.85    148198
Extremely Early Response       0.00      0.00      0.00        83
 Extremely Late Response       0.00      0.00      0.00       555
           Late Response       0.00      0.00      0.00      5005
        On Time Response       0.

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199760 entries, 1016517770 to 1016684613
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   dept                        199760 non-null  object
 1   call_reason                 199760 non-null  object
 2   source_id                   199760 non-null  object
 3   council_district            199760 non-null  int64 
 4   resolution_days_due         199760 non-null  int64 
 5   district_0                  199760 non-null  uint8 
 6   district_1                  199760 non-null  uint8 
 7   district_2                  199760 non-null  uint8 
 8   district_3                  199760 non-null  uint8 
 9   district_4                  199760 non-null  uint8 
 10  district_5                  199760 non-null  uint8 
 11  district_6                  199760 non-null  uint8 
 12  district_7                  199760 non-null  uint8 
 13  district_8      

In [12]:
X_train.drop(columns=['dept', 
                 'call_reason', 
                 'source_id' ], inplace=True)

In [13]:
X_validate.drop(columns=['dept', 
                 'call_reason', 
                 'source_id' ], inplace=True)

In [14]:
X_test.drop(columns=['dept', 
                 'call_reason', 
                 'source_id' ], inplace=True)

# Split Data

### iteration 1 

In [15]:
# create the model object
generic_clf = RidgeClassifier(random_state=123)

In [16]:
# fit to train only
generic_clf.fit(X_train, y_train.level_of_delay)
y_pred = generic_clf.predict(X_train)
# evaluate with score, returns the mean accuracy on the given test data and labels
print('Accuracy of Ridge classifier on training set:', generic_clf.score(X_train, y_train.level_of_delay))

Accuracy of Ridge classifier on training set: 0.8095264317180617


### iteration 2

In [17]:
class_weight_clf = RidgeClassifier(class_weight="balanced", solver='auto', random_state=123)

In [18]:
# fit to train only
class_weight_clf.fit(X_train, y_train.level_of_delay)
y_pred = class_weight_clf.predict(X_train)
# evaluate with score, returns the mean accuracy on the given test data and labels
print('Accuracy of Ridge classifier on training set:', class_weight_clf.score(X_train, y_train.level_of_delay))

Accuracy of Ridge classifier on training set: 0.6045855026031237


### iteration 3

In [19]:
# create the model object
clf2 = RidgeClassifier(solver='sag',random_state=123)

In [20]:
# fit to train only
clf2.fit(X_train, y_train.level_of_delay)
y_pred = clf2.predict(X_train)
# evaluate with score, returns the mean accuracy on the given test data and labels
print('Accuracy of Ridge classifier on training set:', clf2.score(X_train, y_train.level_of_delay))

Accuracy of Ridge classifier on training set: 0.8097867440929115


### iteration 4

In [21]:
clf_cv = RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1])

In [22]:
# fit to train only
clf_cv.fit(X_train, y_train.level_of_delay)
y_pred = clf_cv.predict(X_train)
# evaluate with score, returns the mean accuracy on the given test data and labels
print('Accuracy of Ridge classifier on training set:', clf_cv.score(X_train, y_train.level_of_delay))

Accuracy of Ridge classifier on training set: 0.809536443732479


In [25]:
clf_cv2=RidgeClassifierCV(fit_intercept=True, normalize=True, scoring=None, cv=None, class_weight= 'balanced', store_cv_values=True)

In [27]:
# fit to train only
clf_cv2.fit(X_train, y_train.level_of_delay)
y_pred = clf_cv2.predict(X_train)
# evaluate with score, returns the mean accuracy on the given test data and labels
print('Accuracy of Ridge classifier on training set:', clf_cv2.score(X_train, y_train.level_of_delay))

Accuracy of Ridge classifier on training set: 0.6002402883460152


### iteration 5


In [28]:
clf_cv3=RidgeClassifierCV()

In [29]:
# fit to train only
clf_cv3.fit(X_train, y_train.level_of_delay)
y_pred = clf_cv3.predict(X_train)
# evaluate with score, returns the mean accuracy on the given test data and labels
print('Accuracy of Ridge classifier on training set:', clf_cv3.score(X_train, y_train.level_of_delay))

Accuracy of Ridge classifier on training set: 0.809536443732479
